In [11]:
import json
import urllib
import urllib.request
import pandas as pd

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import viridis, Category20
from bokeh.models import ColumnDataSource
from bokeh.models.formatters import DatetimeTickFormatter
from datetime import datetime
from io import BytesIO

from decryptor import Decryptor

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
BASE_URL = "https://tracking2024.vendeeglobe.org/data/"
CONFIG = "tracker_config"
REPORTS = "tracker_reports"
VERSION = datetime.now().strftime("%Y%m%d%H%M%S")

In [4]:

def download_and_decrypt(element):
    url = BASE_URL + element + ".hwx?version=" + VERSION
    request = urllib.request.Request(url,  headers={'User-Agent' : "Browser"})
    encrypted = urllib.request.urlopen(request).read()
    decrypted = Decryptor().decrypt(encrypted)
    return decrypted

In [5]:
XSLT="""<?xml version="1.0"?>
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">

<xsl:template match="/">
<config>
    <xsl:for-each select="//boat">
    <boat>
            <id><xsl:value-of select="@id" /></id>
            <name><xsl:value-of select="@name"/></name>
            <skipper_fname><xsl:value-of select="./crew/navigator/@fname"/></skipper_fname>
            <skipper_lname><xsl:value-of select="./crew/navigator/@lname"/></skipper_lname>
            <skipper_photo><xsl:value-of select="./crew/navigator/@photo"/></skipper_photo>
    </boat>
    </xsl:for-each>
</config>
</xsl:template>
</xsl:stylesheet>
"""
config_xml = download_and_decrypt(CONFIG)
config = pd.read_xml(BytesIO(config_xml.encode("utf-8")), xpath="//boat", stylesheet=XSLT)

In [6]:
reports_json = download_and_decrypt(REPORTS)
r=json.loads(reports_json)
pr=pd.DataFrame(
    [[history_entry['date']] + line for history_entry in r['reports']['history'] for line in history_entry['lines']],
    columns=['date'] + r['reports']['columns'] 
)
pr.boat = pr.boat.astype(int)
pr.date = pd.to_datetime(pr.date)
reports = pd.merge(pr, config, left_on='boat', right_on='id')

In [ ]:
p = figure(width=800, height=600, toolbar_location=None)
boats=reports.query('date == date.max()').sort_values('rank')['boat'][0:20]
r=reports[reports['boat'].isin(boats)]
r['label'] = r['skipper_fname'] + " " + r['skipper_lname']
for (label, group), color in zip(r.groupby('label'), Category20[20]):
    p.line(x=group.date, y=group['dtl'], legend_label=str(label), color=color, line_width=2)

r2 = r.query('date == date.max()')
ls = LabelSet(x='date', y='dtl', text='label', x_offset=5, y_offset=-10, source=ColumnDataSource(r2))
p.add_layout(ls)
p.y_range.flipped = True
p.xaxis.formatter=DatetimeTickFormatter()
p.legend.location = "bottom_left"
show(p)

/var/folders/t5/39yh6wxj7t74pg0tb7hqp6bm0000gp/T/ipykernel_28436/1624360080.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['label'] = r['skipper_fname'] + " " + r['skipper_lname']
/Users/jmones/.pyenv/versions/3.9.15/lib/python3.9/site-packages/bokeh/util/serialization.py:243: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))
